# Binance用オーダー発行API

`BinanceMarket`は、通過ペアー毎に以下の機能を提供します。

* 蓄積用データベースの生成（Sqlite)
* 約定データのダウンロードと更新
  * 過去データのダウンロード(Binance Webサイト)
  * リアルタイム更新(WebSocket)
* 約定データのDBからの取得と足の作成
  * 任意の期間の足の作成(`ohlcv`メソッド)
  * VAP(Value At Price)の計算(`vap`メソッド)
* オーダーの発行・キャンセル（REST API)　KEYとSECRETの設定が必要です。

また通過ペアーの設定は`BinanceConfig`クラスに定義してあります。パラーメータを修正することで任意の通過ペアーに対応可能ですが、テスト完了して提供しているものは以下になります。
* `BinanceConfig.BTCUSDT` 本番用BTCUSDTペアー（現物）
* `BinanceConfig.TEST_BTCUSDT`　テストネット用BTCUSDTペアー（現物）

## 本サンプルの機能

本サンプルでは以下を行います。

* オーダーの発行(Limit, Market)
* オーダーの検索
* オーダーのキャンセル
* 資産残高の確認

### 利用している関数
* オーダー発行
  * `limit_order(side, price, size)` リミットオーダーの発行
  * `market_order(side, size)`　マーケットオーダーの発行
* オーダー状況確認
  * `order_status`　オーダーステータスの確認（約定済みも含めた直近のもの）
  * `open_orders`　未約定一覧
* オーダーキャンセル
  * `cancel_order(order_id)`　指定したIDのオーダキャンセル
  * `cancel_all_orders()`　未約定のオーダーのキャンセル
* アカウント情報
  * `account`　アカウント情報（残高など）

## API KEYについて（注：まずはテストネットで試してください）
動作させるためにはAPI KEYが必要です。
ローカル環境では、以下の値を環境変数に指定してください。
* BINANCE_API_KEY   APIキー
* BINANCE_API_SERECT  シークレット

Google Clabでは、userdataに設定してください（Colabのページの左側ツールバーにある🔑マークをクリック）。

## 必要ライブラリのインストール

`rbot`がBotFrameWorkの本体です。必要なライブラリを合わせてインストールします。インストール後ランタイムの再起動が必要になる場合があります。

In [1]:
# 必要ライブラリのインストール
! pip install --upgrade pip

# rbotがbot frameworkになります。PyPiに登録されているので、pipでインストールできます。
# まだ開発初期段階なので、インターフェースが変わる可能性があります。そのためバージョン指定しています。
! pip install --upgrade rbot==0.2.6


In [2]:
# 必要ライブラリのインストール。環境によっては要・不要があるので適宜修正してください。
! pip install json2html

## Python コード本体

### 必要ライブラリのインポート

In [1]:
import rbot
# BinanceMarketクラスは、BinanceのAPIを利用して、取引所の情報を取得するクラス
from rbot import BinanceMarket

# BinanceConfigクラスは、BinanceMarketクラスのコンストラクタに渡す設定クラス
from rbot import BinanceConfig

from rbot import init_log, init_debug_log
#init_debug_log()

print("rbot version =", rbot.__version__)

rbot version = 0.2.6


In [2]:
# 必要ライブラリーのインポート
from json2html import *
from IPython.display import HTML

### BinanceMarketオブジェクトの生成

引数にBinanceConfigを設定してオブジェクトを生成します。

In [3]:
# BinanceMarketクラスのインスタンスを生成(テストネットのBTCUSDT現物取引の設定)
config = BinanceConfig.TEST_BTCUSDT

# Google Colabの場合は、userdataからAPIキーを取得
# 通常は、環境変数に設定してください。
if 'google.colab' in str(get_ipython()):
    from google.colab import userdata
    # Google Colabの場合は、APIキーを入力してください。
    config.api_key = userdata.get("BINANCE_API_KEY")
    config.api_secret = userdata.get("BINANCE_API_SECRET")

# json2htmlを利用して、設定情報を表示
HTML(json2html.convert(config.__str__()))

exchange_name,BN
trade_category,TESTSPOT
trade_symbol,BTCUSDT
home_currency,USDT
foreign_currency,BTC
testnet,True
rest_endpoint,https://testnet.binance.vision
public_ws_endpoint,wss://testnet.binance.vision/ws
private_ws_endpoint,wss://testnet.binance.vision/ws
history_web_base,https://data.binance.vision/data/spot/daily/trades
new_order_path,/api/v3/order


In [4]:
# BinanceMarketクラスのインスタンスを生成(BTCUSDT現物取引の設定)
market = BinanceMarket(config)

# BinanceMarketのインスタンスを表示すると格納されているデータの情報が表示されます。
market

start,end
1700846305680000,1700846442059000
"""2023-11-24T17:18:25.680000""","""2023-11-24T17:20:42.059000"""
days=,0


In [5]:
from time import sleep

# WebSocketを利用して、取引所の情報を取得する
market.start_market_stream()

sleep(1) # データが揃うまで少し待つ

In [21]:
# 一旦　過去のオーダーを全部キャンセルする
market.cancel_all_orders()

[{"symbol":"BTCUSDT","create_time":1700846835380000,"status":"Canceled","order_id":"7580784","client_order_id":"NufgEc7IitD1fKDb8MT631","order_side":"Buy","order_type":"Limit","order_price":"37270.00000000","order_size":"0.00100000","remain_size":"0.00100000","transaction_id":"","update_time":0,"execute_price":"0.0","execute_size":"0.0","quote_vol":"0.0","commission":"0.0","commission_asset":"","is_maker":false,"message":"","commission_home":"0.0","commission_foreign":"0.0","home_change":"0.0","foreign_change":"0.0","free_home_change":"0.0","free_foreign_change":"0.0","lock_home_change":"0.0","lock_foreign_change":"0.0"},
 {"symbol":"BTCUSDT","create_time":1700846835380000,"status":"Canceled","order_id":"7580800","client_order_id":"NufgEc7IitD1fKDb8MT631","order_side":"Sell","order_type":"Limit","order_price":"38270.01000000","order_size":"0.00100000","remain_size":"0.00100000","transaction_id":"","update_time":0,"execute_price":"0.0","execute_size":"0.0","quote_vol":"0.0","commission"

## オーダー発行テスト

### Limit order

In [22]:
# 現在のアカウント情報を取り出す
account = market.account

HTML(json2html.convert(account.__str__()))

In [23]:
# accountにはすべての通貨の残高が入っているので、BTCとUSDTの残高を表示する
print("BTC 残高 =", account['BTC'])
print("USDT残高 =", account['USDT'])

BTC 残高 = {"asset":"BTC","free":"1.00400000","locked":"0.00000000"}
USDT残高 = {"asset":"USDT","free":"9849.02128000","locked":"0.00000000"}


In [24]:
# 現在の板情報を取り出す
bids, asks, = market.board

bid_edge = bids['price'][0]
ask_edge = asks['price'][0]

In [26]:
# 板より５００ドル安いところへ0.0001BTCの買い注文を出す
buy_limit_order = market.limit_order("BUY", bid_edge - 500, 0.001)

HTML(json2html.convert(buy_limit_order.__str__()))

symbol,create_time,status,order_id,client_order_id,order_side,order_type,order_price,order_size,remain_size,transaction_id,update_time,execute_price,execute_size,quote_vol,commission,commission_asset,is_maker,message,commission_home,commission_foreign,home_change,foreign_change,free_home_change,free_foreign_change,lock_home_change,lock_foreign_change
BTCUSDT,1700846867669000,New,7582199,r9ZdOeLOSwc9FUKVtkZzEM,Buy,Limit,37321.68000000,0.00100000,0.00100000,,0,0.0,0.0,0.0,0.0,,False,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
# 未約定のオーダーを表示する。オーダーが１つ増えているはず。
open_order = market.open_orders

HTML(json2html.convert(open_order.__str__()))

symbol,create_time,status,order_id,client_order_id,order_side,order_type,order_price,order_size,remain_size,transaction_id,update_time,execute_price,execute_size,quote_vol,commission,commission_asset,is_maker,message,commission_home,commission_foreign,home_change,foreign_change,free_home_change,free_foreign_change,lock_home_change,lock_foreign_change
BTCUSDT,1700842360809000,New,7566098,VHHjQJBZmrfdX6yXrE8t0n,Buy,Limit,38187.96000000,0.00100000,0.00100000,,1700842360809000,0.0,0.00000000,0.0,0.0,,True,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
# 板より５００ドル高いところへ0.0001BTCの売り注文を出す
sell_limit_order = market.limit_order("SELL", ask_edge + 500, 0.001)

HTML(json2html.convert(sell_limit_order.__str__()))

symbol,create_time,status,order_id,client_order_id,order_side,order_type,order_price,order_size,remain_size,transaction_id,update_time,execute_price,execute_size,quote_vol,commission,commission_asset,is_maker,message,commission_home,commission_foreign,home_change,foreign_change,free_home_change,free_foreign_change,lock_home_change,lock_foreign_change
BTCUSDT,1700846914038000,New,7582337,BW4WabPclFIlLDgKglrtSc,Sell,Limit,38321.94000000,0.00100000,0.00100000,,0,0.0,0.0,0.0,0.0,,False,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
# 未約定のオーダーを表示する。オーダーが１つ増えているはず（合計２つ）（価格変動があり約定してしまうことがあります。その場合はmarket.order_statusで直近の状況を確かめられます）
open_order = market.open_orders

HTML(json2html.convert(open_order.__str__()))

symbol,create_time,status,order_id,client_order_id,order_side,order_type,order_price,order_size,remain_size,transaction_id,update_time,execute_price,execute_size,quote_vol,commission,commission_asset,is_maker,message,commission_home,commission_foreign,home_change,foreign_change,free_home_change,free_foreign_change,lock_home_change,lock_foreign_change
BTCUSDT,1700842360809000,New,7566098,VHHjQJBZmrfdX6yXrE8t0n,Buy,Limit,38187.96000000,0.00100000,0.00100000,,1700842360809000,0.0,0.00000000,0.0,0.0,,True,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
# 最新の残高を確認。注文中の資産がLockされているはず。
account = market.account
print("BTC 残高 =", account['BTC'])
print("USDT残高 =", account['USDT'])

BTC 残高 = {"asset":"BTC","free":"1.00300000","locked":"0.00100000"}
USDT残高 = {"asset":"USDT","free":"9774.37792000","locked":"74.64336000"}


In [31]:
# 売りオーダーをキャンセルする

id_to_cancel = buy_limit_order[0].order_id

print("キャンセルするオーダーID =", id_to_cancel)

market.cancel_order(sell_limit_order[0].order_id)

キャンセルするオーダーID = 7582199


{"symbol":"BTCUSDT","create_time":1700846927142000,"status":"Canceled","order_id":"7582337","client_order_id":"i7ixXQ6I1cyZABPZSS5lJB","order_side":"Sell","order_type":"Limit","order_price":"38321.94000000","order_size":"0.00100000","remain_size":"0.00100000","transaction_id":"","update_time":0,"execute_price":"0.0","execute_size":"0.0","quote_vol":"0.0","commission":"0.0","commission_asset":"","is_maker":false,"message":"","commission_home":"0.0","commission_foreign":"0.0","home_change":"0.0","foreign_change":"0.0","free_home_change":"0.0","free_foreign_change":"0.0","lock_home_change":"0.0","lock_foreign_change":"0.0"}

In [32]:
# キャンセルしたオーダーがなくなっていることを確認する
open_order = market.open_orders

HTML(json2html.convert(open_order.__str__()))

symbol,create_time,status,order_id,client_order_id,order_side,order_type,order_price,order_size,remain_size,transaction_id,update_time,execute_price,execute_size,quote_vol,commission,commission_asset,is_maker,message,commission_home,commission_foreign,home_change,foreign_change,free_home_change,free_foreign_change,lock_home_change,lock_foreign_change
BTCUSDT,1700842360809000,New,7566098,VHHjQJBZmrfdX6yXrE8t0n,Buy,Limit,38187.96000000,0.00100000,0.00100000,,1700842360809000,0.0,0.00000000,0.0,0.0,,True,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
# 最新の残高を確認。キャンセルされた分の資産のLockが解放される。
account = market.account
print("BTC 残高 =", account['BTC'])
print("USDT残高 =", account['USDT'])

BTC 残高 = {"asset":"BTC","free":"1.00400000","locked":"0.00000000"}
USDT残高 = {"asset":"USDT","free":"9774.37792000","locked":"74.64336000"}


## Market order

In [34]:
# market orderを出す

market_order = market.market_order("BUY", 0.001)   # 0.001BTCを市場価格で買う
market_order

[{"symbol":"BTCUSDT","create_time":1700846935950000,"status":"Filled","order_id":"7582416","client_order_id":"YzpanHRoPf05rY48CqSxQK","order_side":"Buy","order_type":"Market","order_price":"0.00000000","order_size":"0.00100000","remain_size":"0.00000000","transaction_id":"1543194","update_time":0,"execute_price":"37763.11000000","execute_size":"0.00100000","quote_vol":"37.7631100000000000","commission":"0.00000000","commission_asset":"BTC","is_maker":false,"message":"","commission_home":"0.0","commission_foreign":"0.0","home_change":"0.0","foreign_change":"0.0","free_home_change":"0.0","free_foreign_change":"0.0","lock_home_change":"0.0","lock_foreign_change":"0.0"}]

In [35]:
# 直近のオーダーを表示する。status "Filled"のオーダーが１つ追加されているはず。
# なお大きなオーダーを出すと複数に分割される場合がある。

market.order_status

[{"symbol":"BTCUSDT","orderId":7513729,"orderListId":-1,"clientOrderId":"JbeRhtrx4eYTsjPLQK4xAK","price":"37773.99000000","origQty":"0.00100000","executedQty":"0.00100000","cummulativeQuoteQty":"37.77399000","status":"Filled","timeInForce":"GTC","type":"LIMIT","side":"BUY","stopPrice":"0.00000000","icebergQty":"0.00000000","time":1700827718543,"updateTime":1700827806990,"isWorking":true,"workingTime":1700827718543,"origQuoteOrderQty":"0.00000000","selfTradePreventionMode":"NONE"},
 {"symbol":"BTCUSDT","orderId":7513970,"orderListId":-1,"clientOrderId":"xRSJykRtc1na5F7b9cvIsZ","price":"37974.00000000","origQty":"0.00100000","executedQty":"0.00100000","cummulativeQuoteQty":"37.97400000","status":"Filled","timeInForce":"GTC","type":"LIMIT","side":"SELL","stopPrice":"0.00000000","icebergQty":"0.00000000","time":1700827832755,"updateTime":1700829920736,"isWorking":true,"workingTime":1700827832755,"origQuoteOrderQty":"0.00000000","selfTradePreventionMode":"NONE"},
 {"symbol":"BTCUSDT","order

In [36]:
# 最新の残高を確認。約定した分だけBTCが増えて、USDTの残高が減っているはず。
account = market.account
print("BTC 残高 =", account['BTC'])
print("USDT残高 =", account['USDT'])

BTC 残高 = {"asset":"BTC","free":"1.00500000","locked":"0.00000000"}
USDT残高 = {"asset":"USDT","free":"9736.61481000","locked":"74.64336000"}
